In [50]:
######################################################################################################
######################################################################################################
######################################################################################################
# IMPORT LIBRARIES
import time
import sys
import numpy as np
import sys
import math
import os
import json
import csv
import pandas
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score, KFold
from tensorflow.keras import backend as K
import tensorflow as tf
from scipy.stats import spearmanr, pearsonr
import matplotlib.pyplot as plt
from data_preprocess import preprocess
from sklearn.utils import shuffle
import random
from sklearn.preprocessing import MinMaxScaler
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [51]:
sys.executable

'/Users/franciscogrisanti/anaconda3/envs/museam_env/bin/python'

In [55]:
from data_preprocess import preprocess

fasta_file_positive = 'top_10percent.fa'
fasta_file_negative = 'bottom_10percent.fa'

# Preprocess the data
positive_control = preprocess(f'./data/{fasta_file_positive}','./wt_readout.dat')
positive_control = positive_control.one_hot_encode()
positive_control_names = prep.read_fasta_name_into_array()

negative_control = preprocess(f'./data/{fasta_file_negative}','./wt_readout.dat')
negative_control  = prep.one_hot_encode()
negative_control_names = prep.read_fasta_name_into_array()

features = np.append(positive_control['forward'],negative_control['forward'],axis=0)
targets = np.append(np.ones(len(positive_control['forward'])), np.zeros(len(negative_control['forward'])))      
#names = np.append()


In [64]:
######################################################################################################
######################################################################################################
######################################################################################################
# SET FUNCTIONS

# Get dictionary from text file
def train(file_name):
    dict = {}
    with open(file_name) as f:
        for line in f:
           (key, val) = line.split()
           dict[key] = val

    # change string values to integer values
    dict["filters"] = int(dict["filters"])
    dict["kernel_size"] = int(dict["kernel_size"])
    dict["epochs"] = int(dict["epochs"])
    dict["batch_size"] = int(dict["batch_size"])
    dict["validation_split"] = float(dict["validation_split"])    
    return dict

def run_model(argv = None):
    if argv is None:
        argv = sys.argv
        #input argszw
        fasta_file = argv[1]
        #e.g. sequences.fa
        readout_file = argv[2]
        #e.g. wt_readout.dat
        parameter_file = argv[3]
        #e.g. parameter1.txt

    ## excute the code
    start_time = time.time()

    parameters = train(parameter_file)

    cros_eval(parameters,fasta_file,readout_file)

    # reports time consumed during execution (secs)
    print("--- %s seconds ---" % (time.time() - start_time))


def create_model(dim_num):

    # Modified custom metric functions
    def coeff_determination(y_true, y_pred):
        SS_res =  K.sum(K.square( y_true-y_pred ))
        SS_tot = K.sum(K.square(y_true - K.mean(y_true)))
        return (1 - SS_res/(SS_tot + K.epsilon()))
    def spearman_fn(y_true, y_pred):
        return tf.py_function(spearmanr, [tf.cast(y_pred, tf.float32),tf.cast(y_true, tf.float32)], Tout=tf.float32)
    
    #deepsea arquitecture
    model = tf.keras.Sequential()
    #First Conv1D
    model.add(tf.keras.layers.Conv1D(filters=320, 
                 kernel_size=8, 
                 input_shape=(dim_num[1],dim_num[2])))

    model.add(tf.keras.layers.MaxPooling1D(pool_size=4,strides=4))
    model.add(tf.keras.layers.Dropout(rate=0.20))
    #Second Conv1D
    model.add(tf.keras.layers.Conv1D(filters=480, 
                 kernel_size=8))
    model.add(tf.keras.layers.MaxPooling1D(pool_size=4,strides=4))
    model.add(tf.keras.layers.Dropout(rate=0.20))
    #Third Conv1D
    model.add(tf.keras.layers.Conv1D(filters=960, 
                 kernel_size=8))
    model.add(tf.keras.layers.Dropout(rate=0.50))
    #Dense Layer
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(925, activation='relu'))
    #Output Layer
    model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

    model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                  optimizer='adam', 
                  metrics=['accuracy',coeff_determination, spearman_fn])
    
    model.summary()

    return model


def cros_eval(parameters,fasta_file_positive,fasta_file_negative):
    # Preprocess the data
    positive_control = preprocess(f'./data/{fasta_file_positive}','./wt_readout.dat')
    positive_control = prep.one_hot_encode()
    positive_control_names = prep.read_fasta_name_into_array()

    negative_control = preprocess(f'./data/{fasta_file_negative}','./wt_readout.dat')
    negative_control  = prep.one_hot_encode()
    negative_control_names = prep.read_fasta_name_into_array()

    features = np.append(positive_control['forward'],negative_control['forward'],axis=0)
    targets = np.append(np.ones(len(positive_control['forward'])), np.zeros(len(negative_control['forward'])))      
    names = np.append(positive_control_names, negative_control_names, axis=0).tolist()



    dim_num = fw_fasta.shape


    features_shuffle, target_shuffle, names_shuffle = shuffle(features, targets, names, random_state=seed)

    target_shuffle = np.array(target_shuffle)

    #initialize metrics to save values 
    metrics = []

    #Provides train/test indices to split data in train/test sets.
    kFold = StratifiedKFold(n_splits=10)
    ln = np.zeros(len(target_shuffle))
        
    pred_vals = pandas.DataFrame()

    Fold=0
    model_name = parameters['model_name']

    for train, test in kFold.split(ln, ln):
        model = None
        model = create_model(dim_num)

        fwd_train = forward_shuffle[train]
        fwd_test = forward_shuffle[test]
        

        y_train = target_shuffle[train]
        y_test = target_shuffle[test]

        names_train = names_shuffle[test]
        names_test = names_shuffle[test]

        history = model.fit(fwd_train, y_train, epochs=parameters['epochs'], batch_size=parameters['batch_size'], validation_split=parameters['validation_split'])
        
        history2 = model.evaluate(fwd_test, y_test)

        pred = model.predict(fwd_test)

        metrics.append(history2)

        pred = np.reshape(pred,len(pred))

        temp = pandas.DataFrame({'sequence_names':np.array(names_test).flatten(),
                                     'true_vals':np.array(y_test).flatten(),
                                     'pred_vals':np.array(pred).flatten()})
        temp['Fold'] = Fold

        Fold=Fold+1

        pred_vals = pred_vals.append(temp,ignore_index=True)

    
    pred_vals.to_csv(f'./outs/metrics/{model_name}.csv')

    print('[INFO] Calculating 10Fold CV metrics')   
    g1 = []
    g2 = []
    g3 = []
    for i in metrics:
        loss, r_2, spearman_val = i
        g1.append(loss)
        g2.append(r_2)
        g3.append(spearman_val)

    print(g2)
    print(g3)
    print('seed number = %d' %seed)
    print('Mean loss of 10-fold cv is ' + str(np.mean(g1)))
    print('Mean R_2 score of 10-fold cv is ' + str(np.mean(g2)))
    print('Mean Spearman of 10-fold cv is ' + str(np.mean(g3)))

    metrics_dataframe = pandas.DataFrame({"mean_loss":[np.mean(g1)],
                                          "R_2":[np.mean(g2)],
                                          "Spearman":[np.mean(g3)]})


    metrics_dataframe.to_csv(f'./outs/metrics/{model_name}_CV_metrics.csv')

In [88]:
!python deepsea_classification.py top_10percent.fa bottom_10percent.fa parameters/parameters_deepsea.txt > outs/logs/deepsea.out 

2021-03-09 11:33:49.486729: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-03-09 11:33:49.486910: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-03-09 11:33:49.643305: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
